# CityBikes

Send a request to CityBikes for the city of your choice. 

I submitted this URL via postman:

In order to find the below I used the URL via browser to play around with the JSON with browser filtering.

In [1]:
import requests
import pandas as pd
import json

In [2]:
#chicago network url
url = "http://api.citybik.es/v2/networks/bay-wheels"

response = requests.get(url)
#validate successful response code
print(response)

<Response [200]>


In [3]:
#Testing out what a JSON response, and traversing data points in the response
response_json = json.loads(response.text)
country = response_json['network']['location']['country']
city = response_json['network']['location']['city']
station  = response_json['network']['stations'][0]
uid  = response_json['network']['stations'][0]['id']
print(f"city:\n {city}, country: {country}, id:{uid} station details in json: \n{station}")

city:
 San Francisco Bay Area, CA, country: US, id:d0e8f4f1834b7b33a3faf8882f567ab8 station details in json: 
{'empty_slots': 14, 'extra': {'ebikes': 0, 'has_ebikes': True, 'last_updated': 1703646183, 'payment': ['key', 'creditcard'], 'payment-terminal': True, 'rental_uris': {'android': 'https://sfo.lft.to/lastmile_qr_scan', 'ios': 'https://sfo.lft.to/lastmile_qr_scan'}, 'renting': 1, 'returning': 1, 'slots': 15, 'uid': 'fd89514c-f878-4cd5-8113-8e5beead44de'}, 'free_bikes': 1, 'id': 'd0e8f4f1834b7b33a3faf8882f567ab8', 'latitude': 37.849735, 'longitude': -122.270582, 'name': 'Harmon St at Adeline St', 'timestamp': '2023-12-27T03:04:49.323000Z'}


In [4]:
# Assuming 'response_json' is your JSON data
stations = response_json.get("network", {}).get("stations", [])

for station in stations:
    # station's properties
    station_name = station.get("name", "N/A")
    station_lat = station.get("latitude",0.00)
    station_lon = station.get("longitude",0.00)
    station_id = station.get("id")
    station_num_bike = station.get("free_bikes","N/A")
    station_slots = station.get("empty_slots")
    total_bikes = station_num_bike + station_slots
    
    # Add more attributes as needed
    #Print all stations to view output:
    #print(f"Station Name: {station_name}, Lat: {station_lat}, Lon:{station_lon}, number of bikes: {station_num_bike}, empty slots: {station_slots}, tot bikes: {total_bikes}, id:{station_id}")


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [7]:
#attempt to return results on if the server returns success
if response.status_code == 200:
    data = response.json()
    #If the key "network" is present, it returns its value; otherwise, it returns an empty dictionary {}.
    location = data.get("network", {}).get("location", {})
    #stations = data.get("network", {}).get("stations", {})   
    city = location.get("city")
    latitude = location.get("latitude")
    longitude = location.get("longitude")
    uid = data.get("network", {}).get("stations", [{}])[0].get("id", None)
    #bikes = stations.get("free_bikes")
    bikes = data.get("network", {}).get("stations", [{}])[0].get("free_bikes", None)
    
    # Print values
    print(f"City: {city} \nLatitude: {latitude} \nLongitude: {longitude} \nBikes: {bikes}\nID:{uid}")
else:
    print(f"Error web response code: {response.status_code}")


City: San Francisco Bay Area, CA 
Latitude: 37.7141454 
Longitude: -122.25 
Bikes: 11
ID:d0e8f4f1834b7b33a3faf8882f567ab8


In [5]:
# Return ALL Results
if response.status_code == 200:
    data = response.json()
    counter = 0    
    # Check if "network" and "stations" keys are present in the response
    network_data = data.get("network", {})
    stations_list = network_data.get("stations", [])
    #empty list to plug into the DF
    stations_info = []
    
    # Loop through stations and extract fields
    for station in stations_list:
        location = network_data.get("location", {})
        city = location.get("city")
        uid = station.get("id", None)
        latitude = station.get("latitude")
        #noticed the lat/long was all the same so it was grabbing the network's latlong not the stations
        #longitude = location.get("longitude")
        longitude = station.get("longitude")
        bikes = station.get("free_bikes", None)
        empty_slots = station.get("empty_slots")
        total_bikes = bikes + empty_slots

        # Append information to the list
        stations_info.append({"City": city, "Latitude": latitude, "Longitude": longitude, "Free_Bikes": bikes, "Empty_Slots": empty_slots, "Total_Bikes": total_bikes, "Id": uid,"Station_No": counter})

        counter += 1
    # Create a DataFrame from dictionary
        # Print values for each station
        #print(f"City: {city} \nLatitude: {latitude} \nLongitude: {longitude} \nBikes: {bikes}\n---")
else:
    print(f"Error Response Code: {response.status_code}")

Put your parsed results into a DataFrame.

In [6]:
df_bikes = pd.DataFrame(stations_info)

In [14]:
#Need to differentiate lat/lon of bikes vs POI
df_bikes.rename(columns={'Latitude': 'Bike_Lat'}, inplace=True)
df_bikes.rename(columns={'Longitude': 'Bike_Lon'}, inplace=True)

In [15]:
df_bikes.head()

,Station_No,Station_ID,City,Bike_Lat,Bike_Lon,Free_Bikes,Empty_Slots,Total_Bikes
0,0,d0e8f4f1834b7b33a3faf8882f567ab8,"San Francisco Bay Area, CA",37.849735,-122.270582,1,14,15
1,1,983514094dd808b1604da2dcfc2d09af,"San Francisco Bay Area, CA",37.336188,-121.889277,6,5,11
2,2,da17603652106fda93da4e255a5b0a22,"San Francisco Bay Area, CA",37.322125,-121.881090,16,7,23
3,3,7a21c92b3b4cd2f7759107b4fdebf869,"San Francisco Bay Area, CA",37.323678,-121.874119,7,7,14
4,4,ce34d38fb230a23c1ced12d1e16df294,"San Francisco Bay Area, CA",37.325998,-121.877120,4,23,27


In [16]:
# New column order
column_order = ['Station_No','Station_ID', 'City', 'Bike_Lat','Bike_Lon', 'Free_Bikes', 'Empty_Slots','Total_Bikes']

# Reorder columns
df_bikes = df_bikes[column_order]

# View Modified DataFrame
df_bikes.head()

,Station_No,Station_ID,City,Bike_Lat,Bike_Lon,Free_Bikes,Empty_Slots,Total_Bikes
0,0,d0e8f4f1834b7b33a3faf8882f567ab8,"San Francisco Bay Area, CA",37.849735,-122.270582,1,14,15
1,1,983514094dd808b1604da2dcfc2d09af,"San Francisco Bay Area, CA",37.336188,-121.889277,6,5,11
2,2,da17603652106fda93da4e255a5b0a22,"San Francisco Bay Area, CA",37.322125,-121.881090,16,7,23
3,3,7a21c92b3b4cd2f7759107b4fdebf869,"San Francisco Bay Area, CA",37.323678,-121.874119,7,7,14
4,4,ce34d38fb230a23c1ced12d1e16df294,"San Francisco Bay Area, CA",37.325998,-121.877120,4,23,27


In [17]:
df_bikes.columns

Index(['Station_No', 'Station_ID', 'City', 'Bike_Lat', 'Bike_Lon',
       'Free_Bikes', 'Empty_Slots', 'Total_Bikes'],
      dtype='object')

In [18]:
#export the dataframe to be accessed in a different notebook
df_bikes.to_csv('..\data\df_bikes.csv', index=False)

In [53]:
#32 character unique Bike Station ID
print(df_bikes['Bike_Id'].str.len())

0      32
1      32
2      32
3      32
4      32
       ..
549    32
550    32
551    32
552    32
553    32
Name: Bike_Id, Length: 554, dtype: int64
